<a href="https://colab.research.google.com/github/SilverQ/Multi-Label-Text-Classification-master/blob/master/MultiLabelCNN_google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
base_path = './gdrive/My Drive/Colab Notebooks/multilabelcnn/'

In [0]:
data_in_path = base_path + 'input_data/'
test_data_path = base_path + 'test_data/'
data_out_path = base_path + 'result/'
meta_data_path = base_path + 'meta_data/'
vocab_file = meta_data_path + 'vocab.voc'
label_file = meta_data_path + 'labels.pickle'
freq_file = meta_data_path + 'word_freq.pickle'

In [0]:
import os
if not os.path.exists(data_in_path):
    os.makedirs(data_in_path)
if not os.path.exists(test_data_path):
    os.makedirs(test_data_path)
if not os.path.exists(data_out_path):
    os.makedirs(data_out_path)
if not os.path.exists(meta_data_path):
    os.makedirs(meta_data_path)

In [0]:
# import zipfile
# train_zip_ref = zipfile.ZipFile(data_in_path + 'input_data.zip', 'r')

# # train_zip_ref = zipfile.ZipFile("/content/gdrive/My Drive/Colab Notebooks/multilabelcnn/input_data/input_data.zip", 'r')
# train_zip_ref.extractall(data_in_path)
# train_zip_ref.close()

In [0]:
import zipfile

test_zip_ref = zipfile.ZipFile(test_data_path + 'test_data.zip', 'r')
test_zip_ref.extractall(test_data_path)
test_zip_ref.close()

In [0]:
import tensorflow as tf
import pickle
import json
from tqdm import tqdm
from random import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.backend as K
from collections import Counter
import re
import numpy as np
from tensorflow.keras import preprocessing
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend

In [0]:
tf.enable_eager_execution()

In [0]:
tr_file_list = os.listdir(data_in_path)
tr_file_list = [file for file in tr_file_list if file.endswith(".txt")]
# file_list = [file for file in file_list if file.endswith("docs.txt")]

test_file_list = os.listdir(test_data_path)
test_file_list = [file for file in test_file_list if file.endswith(".txt")]
print(tr_file_list, '\n', test_file_list)

['cpc_json_0_3docs_01.txt', 'cpc_json_0_3docs_02.txt', 'w_dhhan_1007_00.txt', 'w_dhhan_1007_01.txt', 'w_dhhan_1007_02.txt', 'w_dhhan_1007_03.txt', 'w_dhhan_1007_04.txt', 'w_dhhan_1007_05.txt', 'w_dhhan_1007_06.txt', 'w_dhhan_1007_07.txt', 'w_dhhan_1007_08.txt'] 
 ['w_dhhan_1007_09.txt', 'w_dhhan_1007_10.txt', 'w_dhhan_1007_11.txt', 'w_dhhan_1007_12.txt', 'w_dhhan_1007_13.txt', 'w_dhhan_1007_14.txt', 'w_dhhan_1007_15.txt']


In [0]:
class Dataset:

    def __init__(self, train_path, test_path, is_shuffle, train_bs, test_bs, epoch, max_length):
        self.train_path = train_path
        self.test_path = test_path
        self.is_shuffle = is_shuffle
        self.train_bs = train_bs
        self.test_bs = test_bs
        self.epoch = epoch
        self.max_length = max_length
        self.special_tokens = ['<PAD>', '<BOS>', '<EOS>', '<UNK>']
        # self.label_path = label_path
        # self.vocab_path = vocab_path

        if not os.path.exists(vocab_file):
            print('No vocabulary.')
            print('Making vocabulary.')
            self.build_vocab_by_patent(vocab_file)
            print('Complete build vocabulary!')

        # print('Loading vocabulary...')
        self.idx2word, self.word2idx = pickle.load(open(vocab_file, 'rb'))
        print('Successfully load vocabulary!')
        self.idx2label, self.label2idx = pickle.load(open(label_file, 'rb'))
        print('Successfully load labels')

    def build_freq(self, word_list):
        word_counts = Counter(word_list)
        # print('word_list: ', len(word_list), word_list)
        # print('word_counts_1: ', len(word_counts), word_counts)
        # print('word_counts_2: ', len(word_counts.most_common()), word_counts.most_common())
        freq = Counter()
        # freq_file = raw_path + '/word_freq.pickle'
        # print(freq_file)
        if os.path.exists(freq_file):
            with open(freq_file, 'rb') as freq_dist_f:
                freq = pickle.load(freq_dist_f)
                print('frequency distribution loaded', len(freq), freq)
        for word, cnt in word_counts.items():
            # print(word, freq[word])
            freq[word] += cnt
            # print(word, freq[word])
        print('freq len: ', len(freq))
        with open(freq_file, 'wb') as freq_dist_f:
            pickle.dump(freq, freq_dist_f)
        return freq

    def build_vocab_by_patent(self, vocab_file):
        error_cnt = 0
        label_list = []
        for file in self.train_path:
            word_list = []
            with open(data_in_path + file, encoding='utf-8') as f:
                for line in tqdm(f):
                    try:
                        # print('line: ', line)
                        patent = json.loads(line)
                        text = re.sub('[-=.#/?:$}(){,]', ' ', patent['title'] + patent['ab'])
                        token = text.split()
                        # token = tokenizer(patent['title'])
                        # print('token: ', token)
                        # doc = en.tokenizer(patent['title']+patent['ab']+patent['cl'])
                        labels = patent['cpc'].split('|')
                        for tok in token:
                            word_list.append(tok.lower())
                        labels = [label[0] for label in labels]
                        for label in labels:
                            if label not in label_list:
                                label_list.append(label)
                    except:
                        error_cnt += 1
                        # print('error: ', line)
            print('\nIn "%s" word_list: %d, error_cnt: %d\n' % (file, len(word_list), error_cnt))
            idx2word = self.build_freq(word_list)
        idx2word = self.special_tokens + [word for word, _ in idx2word.most_common()]
        print('idx2word: ', len(idx2word), idx2word[:10])
        print('idx2label: ', len(label_list), label_list)
        word2idx = {word: idx for idx, word in enumerate(idx2word)}
        label2idx = {label: idx for idx, label in enumerate(label_list)}
        vocab = (idx2word, word2idx)
        label = (label_list, label2idx)
        pickle.dump(vocab, open(vocab_file, 'wb'))
        pickle.dump(label, open(label_file, 'wb'))

    def text_to_sequence(self, text_list):
        sequences = []
        for text in text_list:
            sequences.append([self.word2idx[word] for word in text if word in self.word2idx.keys()])
        return sequences

    def sequence_to_text(self, sequence):
        return [self.idx2word[idx] for idx in sequence if idx != 0]

    def read_lines(self, indices, path):
        line_count = 0
        texts = []
        labels = []

        with open(path, encoding='utf-8') as f:
            for line in f:
                if line_count in indices:
                    try:
                        patent = json.loads(line)
                        text = re.sub('[-=.#/?:$}(){,]', ' ', patent['title'] + patent['ab'])
                        label = patent['cpc'].split('|')
                        texts.append(text.lower().split())
                        labels.append(list(set([cpc[0] for cpc in label])))
                    except:
                        print(line)
                        print(line_count)
                line_count += 1
        return texts, labels

    def create_multiplehot_labels(self, labels_index):
        labels = []
        # print(len(label))
        for batch in labels_index:
            label = [0] * len(self.label2idx)
            # print(item)
            for cpc in batch:
                label[self.label2idx[cpc]] = 1
            labels.append(label)
        # print('label_repr: ', labels)
        return labels

    def data_generator(self, is_train):
        if is_train:
            batch_size = self.train_bs
            is_shuffle = self.is_shuffle  # 셔플을 여기서 해줘야해. 밖에서는 느려
            file_list = tr_file_list
            path = data_in_path
        else:
            batch_size = self.test_bs
            is_shuffle = False
            file_list = test_file_list
            path = test_data_path
        for file in tqdm(file_list):
            cur_file = path + file
            with open(cur_file, encoding='utf-8') as f:  # 일단 읽어서 길이는 알아둔다.
                data_length = len(f.readlines())
                # print('Num of pat: ', data_length)

            indices = list(range(data_length))  # 인덱스를 미리 만들어주는게 제너레이터 사용의 핵심.
            if is_shuffle:
                shuffle(indices)  # 셔플할꺼라면 이걸... 내장 라이브러리 random에 있는 함수.

            current_count = 0
            while True:
                if current_count >= data_length:
                    return
                else:
                    target_indices = indices[current_count:current_count + batch_size]
                    texts, labels = self.read_lines(target_indices, cur_file)
                    tokenized_title = texts
                    labels = self.create_multiplehot_labels(labels)
                    indexed_encoder_inputs = self.text_to_sequence(tokenized_title)
                    padded_encoder_inputs = pad_sequences(indexed_encoder_inputs,
                                                          maxlen=self.max_length,
                                                          padding='pre')
                    yield padded_encoder_inputs, labels

    def mapping_fn(self, x, y=None):
        inputs, label = {'x': x}, y
        return inputs, label

    def train_input_fn(self):
        dataset = tf.data.Dataset.from_generator(generator=lambda: self.data_generator(is_train=True),
                                                 output_types=(tf.int64, tf.int64),
                                                 output_shapes=(
                                                     (None, self.max_length),  # 넣어주면 graph그릴때 잘못 들어온 입력을 잡아줄 수 있다.
                                                     (None, None)))  # labels count: unknown
        dataset = dataset.map(self.mapping_fn)
        dataset = dataset.repeat(count=self.epoch)
        return dataset

    def test_input_fn(self):
        dataset = tf.data.Dataset.from_generator(generator=lambda: self.data_generator(is_train=False),
                                                 output_types=(tf.int64, tf.int64),
                                                 output_shapes=((None, self.max_length),
                                                                (None, None)))
        dataset = dataset.map(self.mapping_fn)
        return dataset

    def eval_input_fn(self):
        dataset = tf.data.Dataset.from_generator(generator=lambda: self.data_generator(is_train=False))
        dataset = dataset.map(self.mapping_fn)
        return dataset

In [0]:
dataset = Dataset(train_path=tr_file_list,
                  test_path=test_file_list,
                  is_shuffle=True,
                  train_bs=256,
                  test_bs=128,
                  epoch=10,
                  max_length=30)

3it [00:00, 490.41it/s]
4it [00:00, 828.63it/s]
895it [00:00, 8936.86it/s]

No vocabulary.
Making vocabulary.

In "cpc_json_0_3docs_01.txt" word_list: 382, error_cnt: 0

freq len:  144

In "cpc_json_0_3docs_02.txt" word_list: 599, error_cnt: 0

frequency distribution loaded 144 Counter({'the': 37, 'a': 29, 'and': 14, 'first': 12, 'of': 12, 'to': 11, 'second': 10, 'protective': 8, 'segment': 6, 'headgear': 5, 'an': 5, 'is': 5, 'insert': 5, 'cloth': 5, 'end': 4, 'spherical': 4, 'capsule': 4, 'side': 4, 'wall': 4, 'telescoping': 4, 'member': 4, 'which': 4, 'by': 4, 'cap': 4, 'visor': 4, 'assembly': 3, 'eyewear': 3, 'includes': 3, 'via': 3, 'magnetic': 3, 'with': 3, 'having': 3, 'between': 3, 'garment': 3, 'as': 3, 'or': 3, 'has': 3, 'front': 3, 'layer': 3, 'be': 3, 'upper': 3, 'lower': 3, 'flange': 3, 'bill': 3, 'combination': 2, 'attaches': 2, 'connects': 2, 'finger': 2, 'connector': 2, 'positioned': 2, 'such': 2, 'portions': 2, 'central': 2, 'opening': 2, 'defined': 2, 'joint': 2, 'in': 2, 'square': 2, 'piece': 2, 'it': 2, 'can': 2, 'eye': 1, 'protection': 1, '

100000it [00:12, 8328.40it/s]



In "w_dhhan_1007_00.txt" word_list: 11704864, error_cnt: 42

frequency distribution loaded 232 Counter({'the': 91, 'a': 77, 'and': 38, 'to': 27, 'of': 27, 'first': 26, 'second': 22, 'protective': 16, 'is': 13, 'segment': 12, 'an': 11, 'upper': 11, 'eye': 10, 'includes': 10, 'end': 10, 'insert': 10, 'headgear': 9, 'between': 9, 'by': 9, 'spherical': 8, 'capsule': 8, 'side': 8, 'wall': 8, 'telescoping': 8, 'member': 8, 'which': 8, 'cap': 8, 'shoulder': 8, 'having': 7, 'lower': 7, 'assembly': 6, 'eyewear': 6, 'via': 6, 'magnetic': 6, 'with': 6, 'garment': 6, 'or': 6, 'has': 6, 'front': 6, 'layer': 6, 'flange': 6, 'adjustable': 5, 'connects': 5, 'as': 5, 'be': 5, 'cloth': 5, 'hard': 5, 'torso': 5, 'scye': 5, 'bearing': 5, '18': 5, 'combination': 4, 'that': 4, 'attaches': 4, 'finger': 4, 'connector': 4, 'positioned': 4, 'such': 4, 'portions': 4, 'central': 4, 'opening': 4, 'defined': 4, 'joint': 4, 'from': 4, 'in': 4, 'visor': 4, 'for': 4, 'position': 4, 'brim': 4, '14': 4, 'while': 3, 'at

100000it [00:11, 8420.11it/s]


...


In "w_dhhan_1007_07.txt" word_list: 11756301, error_cnt: 466

frequency distribution loaded 386319 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100000it [00:11, 8625.50it/s]



In "w_dhhan_1007_08.txt" word_list: 11758040, error_cnt: 521

frequency distribution loaded 416693 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



idx2word:  445694 ['<PAD>', '<BOS>', '<EOS>', '<UNK>', 'the', 'a', 'of', 'and', 'to', 'is']
idx2label:  9 ['G', 'A', 'B', 'E', 'Y', 'F', 'D', 'H', 'C']
Complete build vocabulary!
Successfully load vocabulary!
Successfully load labels


In [0]:
hyper_params = {'vocab_size': 50000,     # len(dataset.word2idx)
                'label_size': len(dataset.label2idx),
                'embedding_dimension': 256,
                'teacher_forcing_rate': 0.5,
                'use_attention': True}

In [0]:
def model_fn(features, labels, mode, params):
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT
    # feature['x'] => (bs, 20)

    train_op = features
    loss = features
    predicted_token = features
    embedding_layer = tf.keras.layers.Embedding(params['vocab_size'],
                                                params['embedding_dimension'])(features['x'])  # (bs, 20, EMD_SIZE)

    dropout_emb = tf.keras.layers.Dropout(rate=0.5)(embedding_layer)  # (bs, 20, EMD_SIZE)

    filter_sizes = [3, 4, 5]
    pooled_outputs = []
    for filter_size in filter_sizes:
        conv = tf.keras.layers.Conv1D(
            filters=100,
            kernel_size=filter_size,
            padding='valid',
            activation=tf.nn.relu,
            kernel_constraint=tf.keras.constraints.max_norm(3.))(dropout_emb)  # (bs, 20, 100)
        # 최대 norm 지정, weight clipping이 바로 이 부분

        pool = tf.keras.layers.GlobalMaxPool1D()(conv)  # [(bs, 100), (bs, 100), (bs, 100)]
        pooled_outputs.append(pool)

    h_pool = tf.concat(pooled_outputs, axis=1)  # (bs, 300)

    hidden = tf.keras.layers.Dense(units=250, activation=tf.nn.relu,
                                   kernel_constraint=tf.keras.constraints.max_norm(3.))(h_pool)  # (bs, 200)
    dropout_hidden = tf.keras.layers.Dropout(rate=0.5)(hidden, training=TRAIN)
    # logits = tf.keras.layers.Dense(units=1)(dropout_hidden)  # sigmoid를 해주겠다  # (bs, 1)
    logits = tf.keras.layers.Dense(units=params['label_size'])(dropout_hidden)  # 이렇게하면 one-hot 필요

    if TRAIN:
        global_step = tf.train.get_global_step()
        loss = tf.losses.sigmoid_cross_entropy(labels, logits
                                              #  ,
                                              #  weights=1.0, label_smoothing=0.01
                                               )
        # loss = tf.losses.softmax_cross_entropy(labels, logits)

        train_op = tf.train.AdamOptimizer(0.001).minimize(loss, global_step)

        return tf.estimator.EstimatorSpec(mode=mode, train_op=train_op, loss=loss)

    elif EVAL:
        loss = tf.losses.sigmoid_cross_entropy(labels, logits,
                                               weights=1.0, label_smoothing=0.01)
        pred = tf.nn.sigmoid(logits)
        accuracy = tf.metrics.accuracy(labels, tf.round(pred))
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops={'acc': accuracy})

    elif PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions={
                'prob': tf.nn.sigmoid(logits),
            }
        )
    plot_model(model, to_file=data_out_path + 'model.png')

    return tf.estimator.EstimatorSpec(
        mode=mode,
        train_op=train_op,
        loss=loss,
        predictions={'prediction': predicted_token})


In [0]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

est = tf.estimator.Estimator(model_fn=model_fn,
                             params=hyper_params,
                             model_dir=data_out_path)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './gdrive/My Drive/Colab Notebooks/multilabelcnn/result/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4cba8cc8d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
est.train(dataset.train_input_fn)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./gdrive/My Drive/Colab Notebooks/multilabelcnn/result/model.ckpt.


  0%|          | 0/11 [00:00<?, ?it/s]

INFO:tensorflow:loss = 0.69071525, step = 0
INFO:tensorflow:global_step/sec: 53.143
INFO:tensorflow:loss = 0.0002871324, step = 100 (1.886 sec)
INFO:tensorflow:global_step/sec: 59.5571
INFO:tensorflow:loss = 2.7997885e-05, step = 200 (1.676 sec)
INFO:tensorflow:global_step/sec: 62.8318
INFO:tensorflow:loss = 1.079894e-05, step = 300 (1.592 sec)
INFO:tensorflow:global_step/sec: 63.2697
INFO:tensorflow:loss = 6.0927192e-05, step = 400 (1.584 sec)
INFO:tensorflow:global_step/sec: 63.9761
INFO:tensorflow:loss = 2.1206574e-06, step = 500 (1.561 sec)
INFO:tensorflow:global_step/sec: 64.6174
INFO:tensorflow:loss = 9.138266e-06, step = 600 (1.549 sec)
INFO:tensorflow:global_step/sec: 63.2399
INFO:tensorflow:loss = 5.6354605e-07, step = 700 (1.581 sec)
INFO:tensorflow:global_step/sec: 64.7845
INFO:tensorflow:loss = 5.1075244e-07, step = 800 (1.541 sec)
INFO:tensorflow:global_step/sec: 64.0857
INFO:tensorflow:loss = 3.0499865e-07, step = 900 (1.563 sec)
INFO:tensorflow:global_step/sec: 64.2063
I

KeyboardInterrupt: ignored

In [0]:
valid = est.evaluate(dataset.eval_input_fn)
